<a href="https://colab.research.google.com/github/Edward-Gavin/ml-dl/blob/main/taitanic_lr_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchkeras
!pip install torchmetrics

## 依赖导入

In [ ]:
import numpy as np
import pandas as pd
from torch import nn
import torch
from tqdm import tqdm
import os,sys,time
from torch.utils.data import Dataset,DataLoader,TensorDataset
import matplotlib.pyplot as plt
from torchkeras.metrics import Accuracy
from copy import deepcopy
from torchmetrics.classification import BinaryAUROC

In [ ]:
print(os.getcwd())

/content


## 定义模型

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(15, 20),
            nn.ReLU(),
            nn.Linear(20,15),
            nn.ReLU(),
            nn.Linear(15,20),
            nn.ReLU(),
            nn.Linear(20,15),
            nn.ReLU(),
            nn.Linear(15, 1)
        )
        self.metrics = nn.ModuleList([BinaryAUROC()])

    def forward(self, x):
        return self.model(x)

    def metric_update(self, predictions, labels):
        self.metrics[0].update(predictions, labels)

    def metric_fn(self):
        metrics_outs = [met.compute().item() for met in self.metrics]
        [met.reset() for met in self.metrics]
        return metrics_outs

## 加载数据

In [ ]:
train_d = pd.read_csv("/content/sample_data/taitanic/train.csv")
predict_d = pd.read_csv("/content/sample_data/taitanic/test.csv")

train_data = train_d.head(int(891*0.8))
test_data = train_d.tail(int(891*0.2))

train_data

In [ ]:
test_data

## 查看数据

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
ax = train_data['Survived'].value_counts().plot(kind = 'bar', figsize = (6,4),fontsize=7,rot = 0)
ax.set_ylabel('Counts',fontsize = 7)
ax.set_xlabel('Survived',fontsize = 7)
plt.show()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='png'
age = train_data['Age'].plot(kind='hist', bins=20, figsize=(6,4), fontsize=7, rot=0)
age.set_ylabel('Frequency', fontsize=7)
age.set_xlabel('Age', fontsize=7)
plt.show()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
ax = train_data.query('Survived == 0')['Age'].plot(kind = 'density', figsize = (6,4),fontsize=7)
train_data.query('Survived == 1')['Age'].plot(kind = 'density', figsize = (6,4),fontsize=7)
ax.legend(['Survived==0','Survived==1'],fontsize = 7)
ax.set_ylabel('Density',fontsize = 7)
ax.set_xlabel('Age',fontsize = 7)
plt.show()

## 数据预处理

In [ ]:
def preprocessing(dfdata):

    dfresult= pd.DataFrame()

    #Pclass
    dfPclass = pd.get_dummies(dfdata['Pclass']).astype(float)
    dfPclass.columns = ['Pclass_' +str(x) for x in dfPclass.columns ]
    dfresult = pd.concat([dfresult,dfPclass],axis = 1)

    #Sex
    dfSex = pd.get_dummies(dfdata['Sex']).astype(float)
    dfresult = pd.concat([dfresult,dfSex],axis = 1)

    #Age
    dfresult['Age'] = dfdata['Age'].fillna(0)
    dfresult['Age_null'] = pd.isna(dfdata['Age']).astype(float)

    #SibSp,Parch,Fare
    dfresult['SibSp'] = dfdata['SibSp']
    dfresult['Parch'] = dfdata['Parch']
    dfresult['Fare'] = dfdata['Fare']

    #Carbin
    dfresult['Cabin_null'] =  pd.isna(dfdata['Cabin']).astype(float)

    #Embarked
    dfEmbarked = pd.get_dummies(dfdata['Embarked'],dummy_na=True).astype(float)
    dfEmbarked.columns = ['Embarked_' + str(x) for x in dfEmbarked.columns]
    dfresult = pd.concat([dfresult,dfEmbarked],axis = 1)

    return(dfresult)

In [ ]:
x_train = preprocessing(train_data).values
y_train = train_data[['Survived']].values
dl_train = DataLoader(TensorDataset(torch.tensor(x_train).float(),torch.tensor(y_train).float()), shuffle=True, batch_size=36)

x_test = preprocessing(test_data).values
y_test = test_data[['Survived']].values
dl_test = DataLoader(TensorDataset(torch.tensor(x_test).float(), torch.tensor(y_test).float()), shuffle=False, batch_size=9)

In [ ]:
print(x_train.shape)
print(x_test.shape)

(712, 15)
(178, 15)


In [ ]:
print(dl_train.batch_size)
print(dl_test.batch_size)

36
9


## 训练和测试

In [ ]:
# 模型定义
model = Net()
loss_fn = nn.BCEWithLogitsLoss()
optimizer= torch.optim.Adam(model.parameters(),lr = 0.005)

# metrics


history = {}

epochs = 20


for i in range(epochs):
    # 训练
    model.train()

    train_total_loss = 0
    train_step = 0
    train_auc = 0

    loop = tqdm(enumerate(dl_train), total =len(dl_train),file = sys.stdout)

    print("第{}轮训练开始".format(i + 1))
    for index, data in loop:
        features, labels = data
        preds = model(features)

        model.metric_update(preds, labels)
        metrics_outs = model.metric_fn()

        train_auc += metrics_outs[0]


        # metrics_str = ' '.join(["Metric%d : %.3f" % (i, met) for i, met in enumerate(metrics_outs)])
        # print('=========================Training Epoch: %d %s ======================' % (train_step, metrics_str))

        loss = loss_fn(preds, labels)

        #backward
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_step += 1
        train_total_loss += loss.item()

        if train_step % 20 == 0:
            print("训练次数:{}, train total loss: {}, auc: {}".format(train_step, train_total_loss, train_auc / 20))
            train_auc = 0


    # 测试
    model.eval()

    test_total_loss = 0
    test_step = 0
    test_auc = 0
    loop = tqdm(enumerate(dl_test), total =len(dl_test),file = sys.stdout)
    print("第{}轮测试开始".format(i+1))

    with torch.no_grad():
        for i, batch in loop:
            features, labels = batch
            #forward
            preds = model(features)

            model.metric_update(preds, labels)
            metrics_outs = model.metric_fn()

            test_auc += metrics_outs[0]

            # metrics_str = ' '.join(["Metric%d : %.3f" % (i, met) for i, met in enumerate(metrics_outs)])
            # print('=========================Training Epoch: %d %s ======================' % (test_step, metrics_str))

            loss = loss_fn(preds, labels)

            test_total_loss += loss.item()
            test_step += 1
            if test_step % 20 == 0:
                print("测试次数: {}, test total loss: {}, auc: {}".format(test_step, test_total_loss, test_auc/20))
                test_auc = 0


    print("========================================")

## 评估模型

## 模型预测

In [ ]:
# 数据处理
pred_data = preprocessing(predict_d).values

# 预测
y_pred_probs = torch.sigmoid(model(torch.tensor(pred_data).float())).data
y_pred = torch.where(y_pred_probs>0.5, torch.ones_like(y_pred_probs),torch.zeros_like(y_pred_probs))

In [ ]:
# 预测导出
predictions = y_pred.t().numpy()[0]
predictions = pd.Series(predictions, dtype=np.int64)
output = pd.DataFrame({'PassengerId': predict_d.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


## 模型保存与加载

In [ ]:
# 查看模型参数
print(model.state_dict().keys())

# 1. 保存模型参数
torch.save(model.state_dict(), "./taitanic_lr_model.pt")

model_clone = Net()
model_clone.load_state_dict(torch.load("./taitanic_lr_model.pt", weights_only=True))
a = torch.sigmoid(model_clone.forward(torch.tensor(x_test[0:10]).float())).data
print(a)

# 2. 保存完整模型
torch.save(model, "./taitanic_lr_model_all.pt")
model_clone_2 = torch.load("./taitanic_lr_model_all.pt", weights_only=False)
b = torch.sigmoid(model_clone_2.forward(torch.tensor(x_test[0:10]).float())).data
print(b)